### Importing libraties

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

### Define parameters

In [2]:
filepath = 'bumpgrid.dat.txt'

gamma = 1.4

rho_inf = 1.225 
Mach    = 1.4 
P_inf   = 1.0*1e+5
a_inf   = np.sqrt(gamma*P_inf/rho_inf)  # a_inf = sqrt(gamma p / rho) = 1
u_inf   = Mach*a_inf                   
v_inf   = 0.0       

CFL     = 0.5
tol     = 1e-4
niter   = 5000


### Reading Dataset and creating geometry

In [3]:
data = np.loadtxt(filepath)
data = np.array(data)

[imx, jmx] = data[0, :]
imx = int(imx)
jmx = int(jmx)

X = np.zeros([imx, jmx])
Y = np.zeros([imx, jmx])

for j in range(jmx):
    for i in range(imx):
        X[i, j] = data[j * imx + i + 1, 0]
        Y[i, j] = data[j * imx + i + 1, 1]

In [4]:
xnx = np.zeros([imx + 1, jmx + 1])
xny = np.zeros([imx + 1, jmx + 1])

# i-direction faces
for j in range(1, jmx):
    for i in range(imx):
        xnx[i, j] = Y[i, j] - Y[i, j - 1]
        xny[i, j] = -(X[i, j] - X[i, j - 1])

ynx = np.zeros([imx + 1, jmx + 1])
yny = np.zeros([imx + 1, jmx + 1])

for j in range(jmx):
    for i in range(1, imx):
        ynx[i, j] = Y[i - 1, j] - Y[i, j]
        yny[i, j] = -(X[i - 1, j] - X[i, j])

vol = np.zeros([imx + 1, jmx + 1])

for j in range(1, jmx):
    for i in range(1, imx):
        x_1, y_1 = X[i - 1, j - 1], Y[i - 1, j - 1]
        x_2, y_2 = X[i, j - 1], Y[i, j - 1]
        x_3, y_3 = X[i, j], Y[i, j]
        x_4, y_4 = X[i - 1, j], Y[i - 1, j]
        vol[i, j] = 0.5 * (
            abs((x_2 - x_1) * (y_4 - y_2) - (y_2 - y_1) * (x_4 - x_1)) +
            abs((x_4 - x_3) * (y_2 - y_3) - (y_4 - y_3) * (x_2 - x_3))
        )


### Create functions to convert functions

In [5]:
def conservative_to_primitive(rho, rhou, rhov, rhoE, gamma=1.4):
    u = rhou/rho
    v = rhov/rho
    velocity = np.sqrt(u**2 + v**2)
    
